# QR Algorithm

Ci sono delle proprietà:

- Dato che gli eigenvalue di una matrice tirangolare sono gli elementi sulla diagonale, questo algoritmo permette di trovarli.
- Similarity transformation preserva gli eigenvalue.

La matrice A viene **decomposta** in A = QR dove Q è ortogonale e R è triangolare superiore.

## Gram-Schmidt
Come prima implementazione proviamo con l'algoritmo di **Gram-Schmidt**


In [2]:
import numpy as np
from itertools import product
from math import copysign, hypot, sqrt


def print_matrix(matrix):
    s = [[str(e) for e in row] for row in matrix]
    lens = [max(map(len, col)) for col in zip(*s)]
    fmt = '\t'.join('{{:{}}}'.format(x) for x in lens)
    table = [fmt.format(*row) for row in s]
    print('\n'.join(table))
    return

In [12]:
def qr_decomp_gram_schmidt(A):
    m, n = A.shape
    rank = np.linalg.matrix_rank(A)

    if rank < n:
        print(f"Il rango della matrice è {rank} il quale è inferiore del numero delle colonne {n}!")
    
    Q = np.zeros((m, n))
    
    for i, column in enumerate(A.T):
        Q[:,i] = column

        for prev in Q.T[:i]:
            Q[:,i] -= (prev @ column) / (prev @ prev) * prev
    
    Q /= np.linalg.norm(Q,axis=0)
    R = Q.T @ A

    return Q, R

In [13]:
# Tests

#TODO: rendere i test non dipendenti da una sola funzione

def test_orthonormality(A):
    Q, _ = qr_decomp_gram_schmidt(A)
    I = Q.T @ Q

    assert len(set(I.shape)) == 1, "L'identità computata deve essere una matrice quadrata!"
    assert np.allclose(I, np.eye(I.shape[0])), "L'identità  computata deve essere una matrice identità!"

def test_upper_triangular(A):
    _, R = qr_decomp_gram_schmidt(A)

    assert len(set(R.shape)) == 1, "La matrice triangolare superiore computata dovrebbe essere una matrice quadrata!"

    for i, row in enumerate(R):
        assert np.allclose(row[:i], 0), f"La riga {i} della matrice triangolare superiore calcolata non è giusta!"

def test_multiplication(A):
    Q,R = qr_decomp_gram_schmidt(A)
    
    assert np.allclose(Q @ R, A), "La moltiplicazione fra le matrici ottenute non restituisce la matrice originaria!"

In [14]:
test_matricies = (
    np.random.random((2, 2)),
    np.random.random((5, 2)),
    np.random.random((5, 5)),
    np.random.random((9, 8)),
)

tests = (
    test_orthonormality,
    test_upper_triangular,
    test_multiplication,
)

for test, matrix in product(tests, test_matricies):
    test(matrix)

print("tutti i test sono superati")

tutti i test sono superati


# Givens Rotation
proviamo ad implementare le givens rotation per migliorare la velocità di esecuzione.

In [25]:
def qr_decomp_givens_rotation(A):

    (num_rows, num_cols) = np.shape(A)
    if num_rows == num_cols: 
        # Initialize Q,R
        # Q = orthogonal matrix
        # R =  upper triangular matrix
        Q = np.identity(num_rows)
        R = np.copy(A)

        # Iterate over lower triangular matrix.
        (rows, cols) = np.tril_indices(num_rows, -1, num_cols)
        #print(rows, cols)
        for (row, col) in zip(rows, cols):

            # Compute Givens rotation matrix and
            # zero-out lower triangular matrix entries.
            if R[row, col] != 0:
                (c, s) = Givens_Rotation_Matrix_Entries(R[col, col], R[row, col])

                G = np.identity(num_rows)
                G[[col, row], [col, row]] = c
                G[row, col] = s
                G[col, row] = -s

                R = np.dot(G, R)
                Q = np.dot(Q, G.T)
                #R = G @ R
                #Q = Q @ G.T
        return Q, R
    else:
        Q = np.eye(num_rows)
        R = np.copy(A)

        rows, cols = np.tril_indices(num_rows, -1, num_cols)
        for (row, col) in zip(rows, cols):
            # If the subdiagonal element is nonzero, then compute the nonzero 
            # components of the rotation matrix
            if R[row, col] != 0:
                r = np.sqrt(R[col, col]**2 + R[row, col]**2)
                c, s = R[col, col]/r, -R[row, col]/r

                # The rotation matrix is highly discharged, so it makes no sense 
                # to calculate the total matrix product
                R[col], R[row] = R[col]*c + R[row]*(-s), R[col]*s + R[row]*c
                Q[:, col], Q[:, row] = Q[:, col]*c + Q[:, row]*(-s), Q[:, col]*s + Q[:, row]*c

        return Q[:, :num_cols], R[:num_cols]



##### Compute matrix entries for Givens rotation. #####

def Givens_Rotation_Matrix_Entries(a, b):
    r = hypot(a, b)
    c = a/r
    s = -b/r

    return (c, s)

In [23]:
def test_orthonormality(A):
    Q, _ = qr_decomp_givens_rotation(A)
    I = Q.T @ Q

    assert len(set(I.shape)) == 1, "L'identità computata deve essere una matrice quadrata!"
    assert np.allclose(I, np.eye(I.shape[0])), "L'identità  computata deve essere una matrice identità!"

def test_upper_triangular(A):
    _, R = qr_decomp_givens_rotation(A)

    assert len(set(R.shape)) == 1, "La matrice triangolare superiore computata dovrebbe essere una matrice quadrata!"

    for i, row in enumerate(R):
        assert np.allclose(row[:i], 0), f"La riga {i} della matrice triangolare superiore calcolata non è giusta!"

def test_multiplication(A):
    Q, R = qr_decomp_givens_rotation(A)
    
    assert np.allclose(Q @ R, A), "La moltiplicazione fra le matrici ottenute non restituisce la matrice originaria!"

In [29]:
test_matricies = (
    np.random.random((3, 3)),
    np.random.random((4, 5)),
    np.random.random((5, 5)),
    np.random.random((9, 8)),
)

tests = (
    test_orthonormality,
    test_multiplication,
)

for test, matrix in product(tests, test_matricies):
    test(matrix)

print("tutti i test sono superati")

tutti i test sono superati


In [26]:
A = np.random.random((4, 4))

Q, R = qr_decomp_givens_rotation(A)

print(A)
print(Q)
print(R)

print(Q @ R)

[[0.39926392 0.77770333 0.86150654 0.99774404]
 [0.54909614 0.96948566 0.71784402 0.08072706]
 [0.58422336 0.64957978 0.68166213 0.5432887 ]
 [0.73168753 0.70449762 0.41615202 0.66714856]]
[[ 0.39224754  0.0863236  -0.82561526 -0.39629477]
 [ 0.          0.97708796  0.          0.21283588]
 [ 0.57395662  0.1263131   0.5642335  -0.57987874]
 [ 0.71882937 -0.14796064  0.          0.67925841]]
[[ 1.01788765e+00  1.18429642e+00  1.02831061e+00  1.18275278e+00]
 [ 5.36515233e-01  9.92219425e-01  8.00293824e-01  1.34919044e-01]
 [-2.62225490e-17 -2.75569064e-01 -3.26656339e-01 -5.17211022e-01]
 [ 1.16867363e-01  1.05207765e-17 -3.01234187e-01 -2.40094428e-01]]
[[0.39926392 0.77770333 0.86150654 0.99774404]
 [0.54909614 0.96948566 0.71784402 0.08072706]
 [0.58422336 0.64957978 0.68166213 0.5432887 ]
 [0.73168753 0.70449762 0.41615202 0.66714856]]


# Unshifted QR similarità di Hessenberg

con questo metodo si prova a creare una matrice di Hessenberg simile a quelle di partenza applicando gli housolder reflector, lasciando invariati gli autovalori

In [35]:
def Givens_Rotation_Matrix_Entries(a, b):
    r = hypot(a, b)
    c = a/r
    s = b/r    
    return c, s

def Hessenberg_reduction(A):
    # mi trovo gli indici della sottodiagonale corrispondente agli elementi da azzerare
    (num_rows, num_cols) = np.shape(A)
    #itero sugli indici appena trovati 
    for col in range(num_cols-2):
        for row in reversed(range(2+col,num_cols)):
            # trova gli indici della matrice di givens unando come pivot [col, col](sempre != 0)
            c, s = Givens_Rotation_Matrix_Entries(A[row-1, col], A[row, col])
            #si costruisce la matrice di givens partendo dall'identità
            G = np.identity(num_rows)
            #assegna gli indici trovati precedentemente alla matrice di givens
            G[row-1, row-1] = c
            G[row, row] = c
            G[row, row-1] = -s
            G[row-1, row] = s
            # annichilisce l'elemento in posizione A[row, col]
            A = np.dot(G,A)
    return A

In [38]:
A = np.random.random((8, 8))
A = Hessenberg_reduction(A)
print_matrix(A)

0.6664054236639507     	0.4031491875068989     	0.32782764353574123    	0.36420353766762437    	0.03330418615541153  	0.09085262860856347   	0.5054743571569639   	0.18543286348977428
1.6947367090643526     	1.428848368410141      	1.5357703776336273     	1.0971377153840893     	0.7920311547526134   	1.4100812453813865    	1.2291123013130032   	0.7190947480403924 
8.404190781803914e-17  	1.1625963748549408     	0.17869585318232378    	0.8994254585108119     	0.6554768523772854   	0.7541827282510013    	0.3525427260932276   	0.3133085031513735 
-8.46594178090078e-18  	4.745871462684786e-18  	0.8879247496304981     	0.28118232065467014    	0.5062742236882527   	0.27425753962820887   	-0.021072978243412144	0.7053769207109983 
8.983188531323567e-18  	-7.947050553981558e-18 	-3.3945901145539136e-18	0.9307472571112205     	0.3556870901120483   	0.5887474538294828    	0.1724106894197273   	0.6646285922154117 
-1.2232380154726125e-18	-1.5674481020747587e-17	2.218469401985897e-18  	-3.4265761475

In [ ]:
test_matricies = (
    hessenberg(np.random.random((3, 3))),
    hessenberg(np.random.random((4, 4))),
    hessenberg(np.random.random((5, 5))),
    hessenberg(np.random.random((9, 9))),
)

tests = (
    test_orthonormality,
    test_multiplication,
)

for test, matrix in product(tests, test_matricies):
    test(matrix)

print("tutti i test sono superati")